**SQLite를 사용해서 LCEL을 만들 수 있다.**

session_id - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자입니다.


connection - 데이터베이스 연결을 지정하는 문자열입니다. 이 문자열은 SQLAlchemy의 create_engine 함수에 전달됩니다.

In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내 이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("안녕 테디, 만나서 반가워. 나도 잘 부탁해!")

# 채팅 메시지 기록의 메시지들
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워. 내 이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!'),
 AIMessage(content='안녕 테디, 만나서 반가워. 나도 잘 부탁해!')]

이제 Chain에 적용

In [3]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지
        ("system", "You are a helpful assistant."),
        # 대화 기록을 위한 Placeholder
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),  # 질문
    ]
)

model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

# 기존 chain
chain = prompt | model | StrOutputParser()



SQLite 가져오는 함수 만들기

In [4]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

SQLite에 대화정보를 조회할 때 참고하는 ID 구성

In [5]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

이제 history chain 구성

In [6]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수를 설정합니다.
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history",  # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields,  # 대화 기록 조회시 참고할 파라미터를 설정합니다.
)

In [7]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

# 질문과 config 를 전달하여 실행합니다.
print(chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 테디야"}, config))

# 후속 질문을 실해합니다.
print(chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config))



안녕하세요, 테디! 만나서 반가워요. 오늘 어떻게 도와드릴까요?
당신의 이름은 테디라고 하셨죠. 맞나요?


구성되어있는 유저 아이디와 대화 아이디가 일치하지 않으면 정보를 가져올 수 없다.

In [8]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'죄송하지만, 저는 당신의 이름을 알 수 없습니다. 어떻게 도와드릴까요?'